In [1]:
import pandas as pd

In [19]:
from sklearn.feature_extraction import DictVectorizer

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
pd.__version__

'1.4.2'

In [6]:
!pip install pyarrow

     |████████████████████████████████| 42.3 MB 9.2 kB/s eta 0:00:01     |███████████████████████████████▊| 41.9 MB 1.3 MB/s eta 0:00:01


In [3]:
base = "https://d37ci6vzurychx.cloudfront.net/trip-data/"
jan_url = base + "yellow_tripdata_2023-01.parquet"
feb_url = base + "yellow_tripdata_2023-02.parquet"

In [4]:
df_jan = pd.read_parquet(jan_url)
print(df_jan.shape)      # (кол-во строк, кол-во столбцов)
print(df_jan.columns)    # список имён, если нужно убедиться вручную

(3066766, 19)
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')


## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?


In [9]:
df_jan["duration"] = (
        pd.to_datetime(df_jan.tpep_dropoff_datetime) -
        pd.to_datetime(df_jan.tpep_pickup_datetime)
).dt.total_seconds() / 60

std_minutes = df_jan["duration"].std()
print(round(std_minutes, 2))

42.59


## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

90%
92%
95%
98%


In [13]:
mask = df_jan['duration'].between(1, 60, inclusive='both')
fraction = mask.mean() 
print(round(fraction * 100, 2), "%")

98.12 %


## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

2
155
345
515
715

In [17]:
# 1. берём только нужные поля и переводим их в str
cat = df_jan[['PULocationID', 'DOLocationID']].astype(str)

# 2. превращаем в список словарей
records = cat.to_dict(orient='records')

# 3. обучаем DictVectorizer
dv = DictVectorizer(sparse=True)
X = dv.fit_transform(records)

X.shape          #  (N строк, 515 столбцов)

(3066766, 518)

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

3.64
7.64
11.64
16.64

In [22]:
df_jan["duration"] = (
        pd.to_datetime(df_jan.tpep_dropoff_datetime) -
        pd.to_datetime(df_jan.tpep_pickup_datetime)
).dt.total_seconds() / 60
df_jan = df_jan[df_jan["duration"].between(1, 60)]

# 3. one-hot для двух локаций
features = df_jan[["PULocationID", "DOLocationID"]].astype(str)
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(features.to_dict(orient="records"))

# 4. обучаем линейную регрессию
y_train = df_jan["duration"].values
model = LinearRegression()
model.fit(X_train, y_train)

# 5. RMSE на трейне
y_pred = model.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)
print(round(rmse, 2))       # → 7.65

7.65


## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

3.81
7.81
11.81
16.81


In [24]:
# ---------- 1. train (январь) ----------
train_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
df_train = pd.read_parquet(train_url)
df_train["duration"] = (
        pd.to_datetime(df_train.tpep_dropoff_datetime) -
        pd.to_datetime(df_train.tpep_pickup_datetime)
).dt.total_seconds() / 60
df_train = df_train[df_train["duration"].between(1, 60)]

dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(df_train[["PULocationID", "DOLocationID"]]
                           .astype(str).to_dict(orient="records"))
y_train = df_train["duration"].values

model = LinearRegression()
model.fit(X_train, y_train)

# ---------- 2. validation (февраль) ----------
val_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"
df_val = pd.read_parquet(val_url)
df_val["duration"] = (
        pd.to_datetime(df_val.tpep_dropoff_datetime) -
        pd.to_datetime(df_val.tpep_pickup_datetime)
).dt.total_seconds() / 60
df_val = df_val[df_val["duration"].between(1, 60)]

X_val = dv.transform(df_val[["PULocationID", "DOLocationID"]]
                     .astype(str).to_dict(orient="records"))
y_val = df_val["duration"].values
rmse_val = mean_squared_error(y_val, model.predict(X_val), squared=False)
print(round(rmse_val, 2))        # → 7.81

7.81
